In [82]:
from pyspark.sql import SparkSession, functions as F

In [83]:
spark = SparkSession \
        .builder.appName("ETL") \
        .getOrCreate()


In [84]:
df_exportacao = spark.read.json("../data/exportacao.json")
df_producao = spark.read.json("../data/producao.json")

In [85]:
df_exportacao = df_exportacao.select("PAISES", "QUANTIDADE_KG", "VALOR_US").distinct() \
    .groupBy("PAISES") \
    .agg(
        F.sum("QUANTIDADE_KG").alias("Quantidade (kg)"),
        F.sum("VALOR_US").alias("Valor (US$)")
    ) \
    .withColumnRenamed("PAISES", "Pais")


In [86]:
df_producao = df_producao.select("PRODUTO", "QUANTIDADE_L").distinct() \
    .withColumnRenamed("PRODUTO", "produto") \
    .withColumnRenamed("QUANTIDADE_L", "quantidade (Litros)") \
    .groupBy("produto") \
    .agg(
        F.sum("quantidade (Litros)").alias("Total_quantidade (Litros)")
        )


In [88]:
df_producao_pandas = df_producao.toPandas()
df_exportacao_pandas = df_exportacao.toPandas()
df_producao_pandas.to_csv("../output/dados_producao.csv", index=False, header=True)
df_exportacao_pandas.to_csv("../output/dados_exportacao.csv", index=False, header=True)